In [155]:
%reset -f
import glob, os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nms import non_max_suppression_fast

In [166]:
basedir = "/home/kalfasyan/data/images/sticky_plates/2020"
date = '20200820'
images_path = f"{basedir}/{date}/plates"
images_filenames = glob.glob(f"{images_path}/**jpg")

In [167]:
def overlay_image_nms(imgpath, plot_orig=True):
    platename = imgpath.split('/')[-1][:-4]

    img = cv2.imread(imgpath)
    red = img[:,:,2].copy() #cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # 
    red = cv2.medianBlur(red, 5)
    edged = cv2.Canny(red, 20,120)

    (cnts,_) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    edged_image = img.copy()
    cv2.drawContours(edged_image, cnts, -1, (0, 255, 0), 1);

#     cv2.imwrite(f"{imgpath[:-4]}_edged.jpg",edged)

    coordinates, contours = [], []
    for cnt in cnts:
        M = cv2.moments(cnt)
        try:
            cx = int(M['m10']/M['m00'])
        except:
            cx = np.nan
        try:
            cy = int(M['m01']/M['m00'])
        except:
            cy = np.nan
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)

        if (14 < area < 900) and (50 < perimeter < 900):
            if np.nan not in [cx, cy]:
                x,y,w,h = cv2.boundingRect(cnt)
                if np.abs(w-h) < 100 and w/h < 5 and h/w < 5:
                    edged_image = cv2.rectangle(edged_image, (x,y), (x+w, y+h), (0,255,0), 2)
                    coordinates.append((x,y,x+w, y+h))
                    contours.append(cnt)

    # load the image and clone it
    boundingBoxes = np.array(coordinates)

    all_bbox_features = []
    bbox_features = {}

    # perform non-maximum suppression on the bounding boxes
    pick, idxs = non_max_suppression_fast(boundingBoxes, 0.15)
    # loop over the picked bounding boxes and draw them
    for p, (startX, startY, endX, endY) in enumerate(pick):
        idx = np.where(np.all(pick[p]==boundingBoxes,axis=1))[0][0]
        cnt = contours[idx]
        bbox_features['plate_name'] = platename
        bbox_features['index'] = p+1
        bbox_features['area'] = cv2.contourArea(cnt)
        bbox_features['perimeter'] = cv2.arcLength(cnt, True)
    #     bbox_features['ellipse'] = cv2.fitEllipse(cnt)
        bbox_features['convexity_bool'] = cv2.isContourConvex(cnt)
    #     bbox_features['hull'] =  cv2.convexHull(cnt)
        _, bbox_features['enclosing_circle_radius'] = cv2.minEnclosingCircle(cnt)
        bbox_features['B'] = img[startY:endY+1, startX:endX+1, 0].mean()
        bbox_features['G'] = img[startY:endY+1, startX:endX+1, 1].mean()
        bbox_features['R'] = img[startY:endY+1, startX:endX+1, 2].mean()
        bbox_features['startX'] = startX
        bbox_features['startY'] = startY
        bbox_features['endX'] = endX+1
        bbox_features['endY'] = endY+1
        bbox_features['Class'] = ''
        all_bbox_features.append(pd.Series(bbox_features))
    for i, (startX, startY, endX, endY) in enumerate(pick):
        cv2.rectangle(img, (startX, startY), (endX, endY), (0, 255, 0), 2)
        cv2.putText(img, f"{i+1}", (startX-20, startY-10), cv2.FONT_HERSHEY_DUPLEX, 1.2, (0,0,250), 4)

    #     for i, c in enumerate(pick):
    #         cv2.putText(img, "{}".format(i + 1), (c[0]-20, c[1]-10), cv2.FONT_HERSHEY_DUPLEX, 1.2, (0,0,250), 4)

    cv2.putText(img, platename, (10, 4150), cv2.FONT_HERSHEY_DUPLEX, 5., (0,0,250), 5)

    cv2.imwrite(f"{imgpath[:-4]}_overlay.jpg",img)

    print(f"{platename}")
    print(f"{len(coordinates)} insects counted in captured image.")
    print(f"{len(pick)} images after non-maximal-suppression\n")

    df = pd.concat(all_bbox_features,axis=1).T
    df.to_csv(f"{imgpath[:-4]}.txt", sep=',')
    
    return img, df

In [168]:
# test_fname = f"{images_path}/Herent_W31_2-90_F10_51 mm_ISO160_1-15 s_incandescent_23.jpg"

all_dfs = []
for img_fname in images_filenames:
    if True:#img_fname == test_fname:
        resimg, df = overlay_image_nms(img_fname, plot_orig=False)
        all_dfs.append(df)

pd.concat(all_dfs).to_csv(f"{images_path}/results_{date}.xlsx", sep=',')

herent_w33_1-90_F10_51 mm_ISO160_1-15 s_IncandescentWB
1012 insects counted in captured image.
884 images after non-maximal-suppression

herent_w33_1-30_F10_51 mm_ISO160_1-15 s_IncandescentWB
647 insects counted in captured image.
581 images after non-maximal-suppression

herent_w33_4-60_F10_51 mm_ISO160_1-15 s_IncandescentWB
774 insects counted in captured image.
682 images after non-maximal-suppression

herent_w33_1-60_F10_51 mm_ISO160_1-15 s_IncandescentWB
750 insects counted in captured image.
680 images after non-maximal-suppression

racour_20200817_C_F10_51 mm_ISO160_1-15 s_IncandescentWB
799 insects counted in captured image.
707 images after non-maximal-suppression

merchtem_20200818_B3_F10_51 mm_ISO160_1-15 s_IncandescentWB
740 insects counted in captured image.
577 images after non-maximal-suppression

merchtem_20200818_M2_F10_51 mm_ISO160_1-15 s_IncandescentWB
682 insects counted in captured image.
495 images after non-maximal-suppression

herent_w33_4-90_F10_51 mm_ISO160_1-

In [37]:
img_fname

'/home/kalfasyan/data/images/sticky_plates/2020/20200806/plates/Herent_W31_1-30_F10_51 mm_ISO160_1-15 s_incandescent_17.jpg'